# Balances

In [ ]:
from datetime import datetime
print(f"Last executed on {datetime.now()}")

In [ ]:
import crypto_tracker

## Binance

In [ ]:
balances_binance = crypto_tracker.get_binance_balances()

## BlockFi

Download from https://app.blockfi.com/settings/reports

In [ ]:
balances_blockfi = crypto_tracker.get_blockfi_balances()

## Nexo.io

Download from https://platform.nexo.io/transactions

In [ ]:
balances_nexo = crypto_tracker.get_nexo_balances()

## Exodus

In the application, go to
- Exodus in menu bar
- Developer
- Export All Transactions


TODO: get old Exodus wallet

In [ ]:
balances_exodus = crypto_tracker.get_exodus()

## Trust Wallet

In [ ]:
bsc = crypto_tracker.scrape_yieldwatch(timeout=10)
balances_bsc = crypto_tracker.bsc_to_balances(bsc)
balances_trust = crypto_tracker.get_bsc_balance()

print("From YieldWatch:\n")
bsc

# Total in euro
Using CoinGecko

In [ ]:
balances = crypto_tracker.combine_balances(
    balances_binance,
    balances_blockfi,
    balances_exodus,
    balances_nexo,
    balances_trust,
    balances_bsc,
)
eur_balances = crypto_tracker.get_balances_in_euro(balances)

In [ ]:
crypto_tracker.save_data(
    balances_binance,
    balances_blockfi,
    balances_exodus,
    balances_nexo,
    balances_trust,
    balances_bsc,
    eur_balances,
    bsc,
)

In [ ]:
sum(eur_balances.values())

In [ ]:
import pandas as pd

pd.set_option("display.float_format", lambda x: "%.5f" % x)
df = pd.DataFrame([eur_balances, balances], index=["Value (€)", "Amount token"]).T
df['price (€)'] = df["Value (€)"] / df['Amount token']
df

# Plots

In [ ]:
import datetime
import json
from pathlib import Path

import crypto_tracker
import matplotlib.pyplot as plt
import pandas as pd


def load_data():
    fnames = sorted(Path(".").glob("*.json"))

    datas = {}
    for fname in fnames:
        with fname.open("r") as f:
            dt = datetime.datetime.strptime(fname.with_suffix("").name, "%Y%m%d-%H%M%S")
            datas[dt] = json.load(f)
    return datas


def get_df(key, datas):
    df = pd.DataFrame(
        [data[key] for data in datas.values()], [date for date in datas.keys()]
    ).sort_index()
    order = df_eur.iloc[-1].sort_values(ascending=False).index
    return df[order]

datas = load_data()
df_eur = get_df("eur_balances", datas)
df_bal = get_df("balances", datas)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
last = df_eur.iloc[-1]
patches, texts, _ = ax.pie(last, labels=last.index, autopct="%1.1f%%")
factor = 100 / last.sum()
legend_labels = [
    f"{coin} - {factor*amount:1.2f}% - €{amount:.2f}"
    for coin, amount in last.items()
]

ax.axis("equal")
plt.tight_layout()
plt.legend(
    patches,
    legend_labels,
    loc="upper left",
    bbox_to_anchor=(-0.25, 1.0),
    fontsize=12,
)
plt.show()

In [ ]:
def get_title(series):
    coin = series.name
    pct_change = 100 * (series.iloc[-1] - series.iloc[0]) / series.iloc[0]
    amount_change = series.iloc[-1] - series.iloc[0]
    return (
        f"{coin} value over time, {pct_change:.1f}%, €{amount_change:.2f}"
        " change since the start"
    )

total = df_eur.sum(axis=1, skipna=True)
total.name = "total"
total.plot(
    title=get_title(total),
    xlabel="Date",
    ylabel="Value (€)",
)
plt.show()

In [ ]:
last_values = df_eur.iloc[-1].sort_values(ascending=False)
top_n = 15
biggest_holdings = last_values.index[:top_n]

for coin in biggest_holdings:
    bal = df_bal[coin]
    eur = df_eur[coin]
    fig0, ax0 = plt.subplots()
    ax1 = ax0.twinx()
    ax1.spines["right"].set_edgecolor("C1")
    ax1.tick_params(axis="y", colors="C1")
    ax1.set_ylabel(f"Amount in {coin}")
    mid, min_, max_ = bal.mean(), bal.min(), bal.max()
    ax1.set_ylim(min(mid * 0.9, min_), max(mid * 1.1, max_))
    eur.plot(
        ax=ax0,
        xlabel="Date",
        ylabel="Value (€)",
        title=get_title(eur)
    )
    bal.plot(ax=ax1, c="C1")
    plt.show()

# DeFi on BSC

In [ ]:
def get_df_wallet(wallet, datas):
    df = pd.DataFrame(
        [data["balances_per_category"][wallet] for data in datas.values()],
        [date for date in datas.keys()],
    ).sort_index()
    return df


df_bsc = get_df_wallet("bsc", datas)

pct_change = 100 * (df_bsc.iloc[-1] - df_bsc.iloc[0]) / df_bsc.iloc[0]
amount_change = df_bsc.iloc[-1] - df_bsc.iloc[0]
best_5 = pct_change.sort_values(ascending=False)[:5]

for coin, pct_change in best_5.iteritems():
    df_bsc[coin].plot(
        xlabel="Date",
        ylabel=f"Amount ({coin})",
        title=f"{coin} amount, {pct_change:.1}% change since beginning",
    )
    plt.show()